In [1]:
using DelimitedFiles
using Plots

### Load Data

In [2]:
Ids = readdlm("ids.txt", String);
movies = readdlm("movies.txt", String);
ratings = readdlm("ratings.txt", ' ', '\r');
nstudents = size(Ids)[1];
nmovies = size(movies)[1];

prob_Z = readdlm("probZ_init.txt");
prob_R = readdlm("probR_init.txt");
dim_Z = size(prob_Z)[1];
dim_R = size(prob_R)[1];

### Part (a) Sanity check

In [3]:
PerRecom = Dict{String, Float64}();

for i in 1:nmovies
    movieName::String = movies[i];
    ct::Int64 = 0;
    nRecom::Int64 = 0;
    
    for j in 1:nstudents
        if ratings[j,i] == 1
            ct += 1;
            nRecom += 1;
        elseif ratings[j,i] == 0
            ct += 1;
        end 
    end
    
    PerRecom[movieName] = nRecom / ct;
end

In [4]:
sortedMovie = sort(collect(PerRecom), by=x->x[2]);
for i in 1:nmovies
    println(sortedMovie[i][1], "\t", sortedMovie[i][2]);
end

The_Last_Airbender	0.2112676056338028
Fifty_Shades_of_Grey	0.36
I_Feel_Pretty	0.38095238095238093
Magic_Mike	0.42857142857142855
Man_of_Steel	0.5257731958762887
The_Shape_of_Water	0.5616438356164384
World_War_Z	0.5617977528089888
Hustlers	0.5652173913043478
Prometheus	0.5844155844155844
Fast_Five	0.625
American_Hustle	0.6274509803921569
Jurassic_World	0.6287425149700598
Once_Upon_a_Time_in_Hollywood	0.6290322580645161
Pitch_Perfect	0.6428571428571429
Fast_&_Furious:_Hobbs_&_Shaw	0.6551724137931034
Star_Wars:_The_Force_Awakens	0.66
Pokemon_Detective_Pikachu	0.6637168141592921
Phantom_Thread	0.6666666666666666
The_Hunger_Games	0.6717948717948717
Manchester_by_the_Sea	0.6808510638297872
Avengers:_Age_of_Ultron	0.6878612716763006
Rocketman	0.6896551724137931
Mad_Max:_Fury_Road	0.6944444444444444
Us	0.6981132075471698
Bridemaids	0.7
The_Farewell	0.7
Chappaquidick	0.7058823529411765
Good_Boys	0.7142857142857143
Terminator:_Dark_Fate	0.723404255319149
Thor	0.7262569832402235
The_Perks_of_Bein

## Part (e) EM Implementation

### The E step

In [21]:
function E_numerator(slice::Array{Any,1}, Z::Array{Float64,2}, R::Array{Float64,2}, z::Int64)::Float64

    prob_cond::Float64 = Z[z];

    for j::Int64 = 1:nmovies

        if slice[j] == 0
            prob_cond *= 1 - R[j,z];
        elseif slice[j] == 1
            prob_cond *= R[j,z];
        else
            continue;
        end

    end
    
    #if (prob_cond < 0)
    #    error("Negative value encountered at E-step (numerator)");
    #end
    
    return prob_cond;
end

E_numerator (generic function with 1 method)

In [6]:
function joint_prob(slice::Array{Any,1}, Z::Array{Float64,2}, R::Array{Float64,2})::Float64
    # Calculate P({R_j = r_j^t}j \in \Omega _t)
    
    prob_sum::Float64 = 0.0;
    
    for i::Int64 = 1:length(Z)

        prob_sum += E_numerator(slice, Z, R, i);
    end

    return prob_sum;
end

joint_prob (generic function with 1 method)

In [7]:
function E_denominator(slice::Array{Any,1}, Z::Array{Float64,2}, R::Array{Float64,2})::Float64
    return joint_prob(slice, Z, R);
end

E_denominator (generic function with 1 method)

In [8]:
function E_estimate(slice::Array{Any,1}, Z::Array{Float64,2}, R::Array{Float64,2}, z::Int64)::Float64
    return E_numerator(slice, Z, R, z) / E_denominator(slice, Z, R);
end

E_estimate (generic function with 1 method)

### The M step

In [22]:
function M_estimateZ(data::Array{Any,2}, Z::Array{Float64,2}, R::Array{Float64,2}, z::Int64)::Float64
    prob::Float64 = 0.0;
    
    for i::Int64 = 1:nstudents
        prob += E_estimate(data[i,:], Z, R, z);
    end
    
    prob /= nstudents;
    
    #if (prob < 0)
    #    error("Negative value encountered at M-step (rhos)");
    #end
        
    return prob;
end

M_estimateZ (generic function with 1 method)

In [10]:
function M_numeratorR(slice::Array{Any,1}, R::Array{Float64,2}, r::Int64, z::Int64, rhos::Array{Float64,1}, t::Int64)::Float64
    if slice[r] == "?"
        return rhos[t] * R[r,z];
    else
        return rhos[t] * slice[r];
    end
end

M_numeratorR (generic function with 1 method)

In [23]:
function M_numerator(data::Array{Any,2}, R::Array{Float64,2}, r::Int64, z::Int64, rhos::Array{Float64,1})::Float64
    
    prob::Float64 = 0.0;
    
    for i::Int64 = 1:nstudents
        increment = M_numeratorR(data[i,:], R, r, z, rhos, i);
        #print(increment, " ");
        prob += increment;
    end
    
    #if (prob < 0)
    #    error("Negative value encountered at M-step (numerator)");
    #end
        
    return prob;
end

M_numerator (generic function with 1 method)

In [24]:
function M_estimateR(data::Array{Any,2}, Z::Array{Float64,2}, R::Array{Float64,2}, r::Int64, z::Int64)::Float64
    
    rhos::Array{Float64,1} = zeros(nstudents);
    for i::Int64 = 1:nstudents
        rhos[i] = E_estimate(data[i,:], Z, R, z);
    end
    
    denominator = sum(rhos);
    numerator = M_numerator(data, R, r, z, rhos);
    
    newR::Float64 = numerator/denominator;
    
    #if (newR < 0)
    #    error("Negative value encountered at M-step (R)");
    #elseif (newR > 1)
    #    println(numerator, " ", denominator);
    #    println(r, " ", z, " ", newR);
    #    error("Invalid probability value at M-step (R)");
    #end

    return newR;
end

M_estimateR (generic function with 1 method)

### log-likelihood

In [13]:
function log_likelihood(data::Array{Any,2}, Z::Array{Float64,2}, R::Array{Float64,2})
    
    L::Float64 = 0.0;
    
    for i::Int64 = 1:nstudents
        L += log(joint_prob(data[i,:], Z, R));
    end
    
    L /= nstudents;
    
    return L;
end

log_likelihood (generic function with 1 method)

### update rules

In [14]:
function update(data::Array{Any,2}, Z::Array{Float64,2}, R::Array{Float64,2})
    
    new_Z::Array{Float64,2} = zeros((dim_Z, 1));
    new_R::Array{Float64,2} = zeros((dim_R, dim_Z));
    
    for z::Int64 = 1:dim_Z
         new_Z[z] = M_estimateZ(data, Z, R, z);
    end
    
    for r::Int64 = 1:dim_R
        for z::Int64 = 1:dim_Z
            new_R[r,z] = M_estimateR(data, Z, R, r, z);
        end
    end
    
    return new_Z, new_R;
end

update (generic function with 1 method)

In [39]:
function iterate(data::Array{Any,2}, initZ::Array{Float64,2}, initR::Array{Float64,2}, max_iter::Int64 = 256)
     
    Ls::Array{Float64,1} = zeros(max_iter+1);
    Ls[1] = log_likelihood(data, initZ, initR);
    
    Z::Array{Float64,2} = deepcopy(initZ);
    R::Array{Float64,2} = deepcopy(initR);
    
    println("L= ", Ls[1]);
    
    for iter::Int64 = 1:max_iter
        #println(Z);
        #println(R[1,:]);
        Z, R = update(data, Z, R);
        Ls[iter+1] = log_likelihood(data, Z, R);
        
        #if iter==1 || iter==16
        #    println("Iter= ", iter, " L= ", Ls[iter+1]);
        #end
        #Z, R = deepcopy(nZ), deepcopy(nR);
    end
    
    return Ls, Z, R;
end

iterate (generic function with 2 methods)

In [40]:
myL, Z, R = iterate(ratings, prob_Z, prob_R);

L= -29.327593818611266


##### Report Ls

In [41]:
myIters = [0, 1, 2, 4, 8, 16, 32, 64, 128, 256];
for it in myIters
     println(it, " iterations, likelihood is ", myL[it+1]);
end

0 iterations, likelihood is -29.327593818611266
1 iterations, likelihood is -18.13928372437194
2 iterations, likelihood is -16.17129923219528
4 iterations, likelihood is -14.941642713922825
8 iterations, likelihood is -14.21071932394614
16 iterations, likelihood is -13.858051333076746
32 iterations, likelihood is -13.763965178021762
64 iterations, likelihood is -13.7398309258038
128 iterations, likelihood is -13.737716834859478
256 iterations, likelihood is -13.737497910546878


### Part (f) Personal movie recommendations

In [42]:
myIndex = findall(id->id=="U08422735", Ids)[1][1];
println("my index in dataset is ", myIndex);

my index in dataset is 74


##### report posterior probability

In [45]:
posterior = zeros(dim_Z);

for z = 1:dim_Z
    posterior[z] = E_estimate(ratings[myIndex,:], Z, R, z);
    println("Z=", z, ", prob is ", posterior[z]);
end

Z=1, prob is 0.063243875578235
Z=2, prob is 0.0
Z=3, prob is 0.9171904701627055
Z=4, prob is 0.019565654259059463


##### Compute expectation

In [55]:
function expectation(slice::Array{Any,1}, Z::Array{Float64,2}, 
                        R::Array{Float64,2}, posterior::Array{Float64,1})::Array{Float64,1}
    
    ratings_expected::Array{Float64,1} = zeros(nmovies);
    
    for l::Int64 = 1:nmovies
        
        # Filter seen movies
        if slice[l] != "?"
            ratings_expected[l] = -1.0;
            continue;
        end
        
        sum::Float64 = 0.0;
        
        for z::Int64 = 1:dim_Z
            sum += posterior[z] * R[l,z];
        end
        
        ratings_expected[l] = sum;
    end
    
    return ratings_expected;
end

expectation (generic function with 2 methods)

In [62]:
my_expectation = expectation(ratings[myIndex,:], Z, R, posterior);
movie_expectation = Dict();
for l = 1:nmovies
    if my_expectation[l] == -1.0
        continue;
    end
    
    movie_expectation[movies[l]] = my_expectation[l];
end

sortedExpectation = sort(collect(movie_expectation), by=x->x[2]);

##### Report the expectations

In [69]:
for pair in sortedExpectation
     println("Movie: ", pair[1], "\n  ", pair[2]);
end

Movie: The_Last_Airbender
  0.210381890523104
Movie: Fifty_Shades_of_Grey
  0.40985445105392376
Movie: Magic_Mike
  0.4347635164340712
Movie: I_Feel_Pretty
  0.457546131906956
Movie: The_Shape_of_Water
  0.5743224843153195
Movie: Hustlers
  0.6117910420083135
Movie: World_War_Z
  0.6326232255696812
Movie: Good_Boys
  0.6571203861956484
Movie: Man_of_Steel
  0.6636840981216302
Movie: Pitch_Perfect
  0.6902822625284639
Movie: Fast_&_Furious:_Hobbs_&_Shaw
  0.6903040530658958
Movie: Bridemaids
  0.7020865014412344
Movie: Once_Upon_a_Time_in_Hollywood
  0.7059194052212779
Movie: Us
  0.715378796629833
Movie: American_Hustle
  0.7309906517890273
Movie: Manchester_by_the_Sea
  0.7349537116228902
Movie: Ex_Machina
  0.7487810938075038
Movie: The_Hunger_Games
  0.7500642474866003
Movie: Prometheus
  0.7605831423173289
Movie: Jurassic_World
  0.7633052802742801
Movie: Mad_Max:_Fury_Road
  0.7702494884520378
Movie: Phantom_Thread
  0.7747747387992352
Movie: Dunkirk
  0.7857997767997764
Movie: Mi